In [87]:
%pip install scikit-learn
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import logging
logging.basicConfig(level=logging.INFO)

### List of functions
----------------------------

1. ridge_regression
2. rank_cross_section_features
3. random_fourier_features
4. ranked_fourier_features

In [3]:
def ridge_regression(
    signals: np.ndarray,
    labels: np.ndarray,
    future_signals: np.ndarray,
    shrinkage_list: np.ndarray,
):
    """Smart Ridge Regression"""
    t_ = signals.shape[0]
    p_ = signals.shape[1]

    matr = signals @ signals.T / t_

    if p_ < t_:
        # this is standard regression
        eigenvalues, eigenvectors = np.linalg.eigh(np.dot(signals.T, signals) / t_)
        means = signals.T @ labels.reshape(-1, 1) / t_
        multiplied = eigenvectors.T @ means
        intermed = np.concatenate(
            [
                (1 / (eigenvalues.reshape(-1, 1) + z)) * multiplied
                for z in shrinkage_list
            ],
            axis=1,
        )
        betas = eigenvectors @ intermed

    else:
        eigenvalues, eigenvectors = np.linalg.eigh(matr)
        means = labels.reshape(-1, 1) / t_
        multiplied = eigenvectors.T @ means
        intermed = np.concatenate(
            [
                (1 / (eigenvalues.reshape(-1, 1) + z)) * multiplied
                for z in shrinkage_list
            ],
            axis=1,
        )
        tmp = eigenvectors.T @ signals
        betas = tmp.T @ intermed

    predictions = future_signals @ betas
    return betas, predictions

In [4]:
def rank_cross_section_features(X: pd.DataFrame, date_ids: np.ndarray):
    """Simple Cross-Section Ranking [-0.5, 0.5]"""
    assert isinstance(X, pd.DataFrame)
    X["date"] = date_ids
    X = X.groupby("date").rank(pct=True, axis=0) - 0.5
    return X

In [5]:
def random_fourier_features(
    x_train: pd.DataFrame, 
    x_test: pd.DataFrame, 
    number_features: int, 
    seed: int
):
    """Clean Random Fourier Feature, see Rahimi 2007."""
    np.random.seed(seed=(seed+1*1e3))
    weights = np.random.normal(loc=0, scale=1, size=[x_train.shape[1], number_features])

    # np.__version__ == 1.26.4
    # XXX np.arange(0.5, 1.1, step=0.1) bugged :(
    gammas = np.random.choice(
        [0.5, 0.6, 0.7, 0.8, 0.9, 1], 
        size=[x_train.train.shape[1], number_features], 
        replace=True
    )
    weights = weights * gammas
    x_train_rff = pd.concat(
        [getattr(np, activation)(x_train @ weights) for activation in ["cos", "sin"]]
        
    )
    
    x_test_rff = pd.concat(
        [getattr(np, activation)(x_test @ weights) for activation in ["cos", "sin"]]
    )

    return x_train_rff, x_test_rff

In [6]:
def ranked_random_fourier_features(
    x_train: pd.DataFrame, 
    x_test: pd.DataFrame, 
    number_features: int, 
    seed: int,
    train_dates: np.ndarray
):
    """Applies Ranking After RFF"""
    x_train_rff, x_test_rff = random_fourier_features(
        x_train=x_train,
        x_test=x_test,
        number_features=number_features,
        seed=seed,
    )
    x_train_rff = rank_cross_section_features(
        x_train_rff,
        date_ids=train_dates
    )
    # XXX Monthly re-training
    x_test_rff = x_test_rff.rank(pct=True, axis=0) - 0.5
    return x_train_rff, x_test_rff

In [22]:
def linear_rolling_window_training(df: pd.DataFrame, rolling_window: int, long_only: bool) -> pd.DataFrame:
    dates = df.date.unique().tolist()
    dates.sort()
    
    start_window = 0
    end_window = rolling_window
    portfolio_returns = []
    while start < len(dates):
        start_time = time.monotonic()
 
        train_dates = dates[start_window:end_window]
        test_date = dates[end_window]

        train = df[df.date.isin(train_dates)].copy()
        test: pd.DataFrame = df[df.date == test_date].copy()

        train.set_index("date", inplace=True)
        test.set_index("date", inplace=True )
        
        y_train = train.pop("r_1")
        y_test = test.pop("r_1")

        train = train.multiply(y_train, axis="index")
        F_train = train.reset_index().groupby("date").sum()
        assert F_train.shape[0] == rolling_window
        model = LinearRegression(fit_intercept=False, positive=long_only)
        model.fit(F_train.values, np.ones(F_train.shape[0])) 

        prediction = test @ model.coef_
        returns = prediction.values.reshape(1,-1) @ y_test
        portfolio_returns.append({
            "date": test_date,
            "return": returns[0]
        })
        start_window += 1
        end_window += 1
        end_time = time.monotonic()
        logging.info(f"[{end_window}/{len(dates)}]\tTraining Time:{end_time-start_time:.2f}s\tPF Return:{returns[0]:3f}")
    return portfolio_returns

In [25]:
def random_features_rolling_window_training(
        df: pd.DataFrame, 
        rolling_window: int,
        number_features: int,
        seed: int
) -> pd.DataFrame:
    dates = df.date.unique().tolist()
    dates.sort()

    start = rolling_window
    i = 0
    portfolio_returns = []
    while start < len(dates):
        
        start_time = time.monotonic()
        train_dates = dates[i:start]
        test_date = dates[start]

        train = df[df.date.isin(train_dates)].copy()
        test: pd.DataFrame = df[df.date == test_date].copy()

        y_train = train.pop("r_1")
        y_test = test.pop("r_1")

        train_dates = train.pop("date")

        x_train_rff, x_test_rff = ranked_random_fourier_features(
            x_train=train,
            x_test=test,
            number_features=number_features, 
            seed=seed,
            train_dates=train_dates
        )

        x_train_rff = x_train_rff.multiply(y_train, axis="index")
        x_train_rff.index = train_dates
        F_train = train.reset_index().groupby("date").sum()
        assert F_train.shape[0] == rolling_window
        model = LinearRegression(fit_intercept=False)
        model.fit(X=F_train, y=np.ones(F_train.shape[0]))
        prediction = x_test_rff @ model.coef_
        returns = prediction.values.reshape(1,-1) @ y_test
        portfolio_returns.append({
            "date": test_date,
            "return": returns[0]
        })
        start += 1
        i += 1
        end_time = time.monotonic()
        logging.info(f"[{start}/{len(dates)}]\tTraining Time:{end_time-start_time:.2f}s\tPF Return:{returns[0]:.3f}")
    return portfolio_returns

## Empirics
Get the data from [Jensen, T. I., Kelly, B., & Pedersen, L. H. (2023).](https://jkpfactors.com/?country=usa&factor=all_factors&weighting=ew)<br>
For simplicity, we will restrict the analysis to large and mega cap stocks only.

In [9]:
jkp = pd.read_pickle("jkp_ranked_large_mega.pickle")
jkp.pop("size_grp")
jkp.pop("id")

0          10006
2          10102
3          10145
4          10153
5          10161
           ...  
2486994    93356
2486996    93369
2487000    93374
2487004    93427
2487006    93436
Name: id, Length: 671743, dtype: int64

In [10]:
jkp.describe()


,date,cowc_gr1a,oaccruals_at,oaccruals_ni,taccruals_at,taccruals_ni,debt_gr3,fnl_gr1a,ncol_gr1a,nfna_gr1a,...,div12m_me,ebitda_mev,eq_dur,eqnpo_12m,eqnpo_me,eqpo_me,ni_me,ocf_me,sale_me,r_1
count,671743,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,...,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000,671743.000000
mean,1996-09-13 06:06:20.749185408,-0.019091,0.002287,-0.019457,-0.002163,-0.020176,0.014176,0.020228,0.053547,-0.011736,...,-0.006512,0.023517,0.019616,0.072738,0.091148,0.093303,0.055811,0.050632,-0.056205,0.006538
min,1963-01-31 00:00:00,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,...,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.998298
25%,1983-06-30 00:00:00,-0.191082,-0.177930,-0.222597,-0.192600,-0.225728,-0.156537,-0.212499,-0.168622,-0.231179,...,-0.326073,-0.162813,-0.173141,-0.158307,-0.036066,-0.003119,-0.124022,-0.124103,-0.271667,-0.047461
50%,1998-02-28 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039233,0.071793,-0.023029,...,0.049470,0.013073,0.032006,0.124471,0.092892,0.067191,0.059565,0.043767,-0.074891,0.005307
75%,2010-07-31 00:00:00,0.139831,0.178554,0.175735,0.187959,0.176757,0.195017,0.246354,0.290834,0.211918,...,0.265673,0.209884,0.222395,0.315263,0.298246,0.284684,0.247790,0.241699,0.143129,0.058897
max,2022-11-30 00:00:00,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.499656,2.997363
std,NaN,0.230908,0.238614,0.254723,0.245843,0.256605,0.241784,0.269545,0.278295,0.264254,...,0.329512,0.236675,0.249492,0.282324,0.245779,0.229699,0.239658,0.234916,0.253979,0.106385


In [11]:
rolling_window = 60
number_random_features = 2000
seed = 1234

In [24]:
msrr_ols = linear_rolling_window_training(df=jkp, rolling_window=rolling_window, long_only=False)

INFO:root:[61/719]	Training Time:0.04s	PF Return:0.886068


AssertionError: 